# 🧬 UFoE Protein Creator Engine + ESMFold Validation

**UFoE Layered Protein: 비율과 층위로 읽는 단백질 구조**

Author: Young Kang (MCWS) | 2026-02

---

### 검증 파이프라인

```
Step 1: Creator Engine — 12칸 시그니처 → 서열 생성 → 내부 3D 시뮬레이션 → FASTA 출력
Step 2: ESMFold — FASTA 입력 → 독립적 3D 구조 예측 → PDB 출력
Step 3: Signature Analysis — ESMFold PDB → 12칸 시그니처 측정 → 입력과 비교
```

> **핵심**: ESMFold는 UFoE를 모릅니다. 곤감리건도 모릅니다. 순전히 자체 학습 데이터로 구조를 예측합니다.
> 그런데도 12칸 기울기가 보존된다면 — UFoE가 ESMFold가 암묵적으로 학습한 물리적 원리를 명시적으로 기술하고 있다는 뜻입니다.

---


## 0. 환경 설정

In [ ]:
# 패키지 설치 (Colab에서 처음 1회만 실행)
!pip install -q biopython matplotlib pandas seaborn

import numpy as np
import math
import json
import os
import time
from collections import defaultdict

import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['font.size'] = 11
import seaborn as sns
import pandas as pd

print("✅ 기본 패키지 로드 완료")
print(f"NumPy: {np.__version__}")

# GPU 확인
import subprocess
result = subprocess.run(['nvidia-smi', '--query-gpu=name,memory.total', '--format=csv,noheader'],
                       capture_output=True, text=True)
if result.returncode == 0:
    print(f"🖥 GPU: {result.stdout.strip()}")
else:
    print("⚠ GPU 미감지 — Runtime > Change runtime type > GPU 선택 필요")


## 1. UFoE 상수 및 핵심 정의

In [ ]:
# ============================================================
# 곤감리건 아미노산 분류
# ============================================================
GON_AAS = ['V', 'I', 'L', 'F', 'W', 'M', 'A']   # 소수성 코어 (7)
GAM_AAS = ['C', 'G', 'P']                          # 구조적 제약 (3)
RI_AAS  = ['S', 'T', 'N', 'Q', 'H', 'Y']          # 극성 접착/허브 (6)
GEON_AAS = ['K', 'R', 'D', 'E']                    # 전하 표면 (4)

TYPE_MAP = {}
for aa in GON_AAS:  TYPE_MAP[aa] = 'Gon'
for aa in GAM_AAS:  TYPE_MAP[aa] = 'Gam'
for aa in RI_AAS:   TYPE_MAP[aa] = 'Ri'
for aa in GEON_AAS: TYPE_MAP[aa] = 'Geon'

TYPES = ['Gon', 'Gam', 'Ri', 'Geon']
ZONES = ['Center', 'Transition', 'Boundary']
TYPE_COLORS = {'Gon': '#2196F3', 'Gam': '#FF9800', 'Ri': '#4CAF50', 'Geon': '#F44336'}

# PDB 1,473개 평균 기준 매트릭스 (Zone-normalized)
BASELINE_MATRIX = {
    ('Center', 'Gon'):    0.4625,  ('Center', 'Gam'):    0.1366,
    ('Center', 'Ri'):     0.2455,  ('Center', 'Geon'):   0.1534,
    ('Transition', 'Gon'):0.4042,  ('Transition', 'Gam'):0.1365,
    ('Transition', 'Ri'): 0.2546,  ('Transition', 'Geon'):0.2048,
    ('Boundary', 'Gon'):  0.2909,  ('Boundary', 'Gam'):  0.1513,
    ('Boundary', 'Ri'):   0.2747,  ('Boundary', 'Geon'): 0.2830,
}

BASELINE_ANTAGONISM = (
    (BASELINE_MATRIX[('Center','Gon')] - BASELINE_MATRIX[('Boundary','Gon')]) -
    (BASELINE_MATRIX[('Center','Geon')] - BASELINE_MATRIX[('Boundary','Geon')])
)

# 4×4 친화도 행렬
AFFINITY = {
    ('Gon','Gon'): 5, ('Gon','Gam'): 2, ('Gon','Ri'): 3, ('Gon','Geon'): 0,
    ('Gam','Gon'): 2, ('Gam','Gam'): 3, ('Gam','Ri'): 4, ('Gam','Geon'): 2,
    ('Ri','Gon'):  3, ('Ri','Gam'):  4, ('Ri','Ri'):  3, ('Ri','Geon'):  4,
    ('Geon','Gon'):0, ('Geon','Gam'):2, ('Geon','Ri'): 4, ('Geon','Geon'):3,
}

# φ 상수
PHI = (1 + math.sqrt(5)) / 2
CENTER_THRESH = 1 / (PHI ** 2)     # ≈ 0.382
TRANS_THRESH = 1 / PHI              # ≈ 0.618

print(f"✅ UFoE 상수 로드")
print(f"  기준 상극축 (PDB 1,473 평균): {BASELINE_ANTAGONISM:.4f}")
print(f"  φ 경계: Center < {CENTER_THRESH:.3f}, Transition < {TRANS_THRESH:.3f}")
print(f"  곤감리건 = 7:3:6:4 = {len(GON_AAS)}:{len(GAM_AAS)}:{len(RI_AAS)}:{len(GEON_AAS)}")


## 2. Creator Engine 핵심 (Stage 1-6)

파이프라인:
```
Stage 1: 상극축 → 12칸 시그니처 생성
Stage 2: 시그니처 → 서열 생성
Stage 3: 소거 필터
Stage 4: 성장-소거 3D 시뮬레이션
Stage 5: 3D 구조 → 12칸 재분석
Stage 6: 자기검증 (순환 닫기)
```


In [ ]:
# ============================================================
# STAGE 1: 상극축 → 12칸 시그니처 생성
# ============================================================
def generate_signature(target_antagonism):
    sig = {}
    for key in BASELINE_MATRIX:
        sig[key] = BASELINE_MATRIX[key]

    delta = target_antagonism - BASELINE_ANTAGONISM
    adjustment = delta / 4.0

    sig[('Center', 'Gon')]    += adjustment
    sig[('Boundary', 'Gon')]  -= adjustment
    sig[('Center', 'Geon')]   -= adjustment
    sig[('Boundary', 'Geon')] += adjustment

    for z in ZONES:
        total = sum(sig[(z, t)] for t in TYPES)
        if abs(total - 1.0) > 0.001:
            sig[(z, 'Ri')] += (1.0 - total)

    for key in sig:
        sig[key] = max(0.01, min(0.99, sig[key]))

    for z in ZONES:
        total = sum(sig[(z, t)] for t in TYPES)
        for t in TYPES:
            sig[(z, t)] /= total

    return sig


# ============================================================
# STAGE 2: 시그니처 → 서열 생성
# ============================================================
def generate_composition(length, signature, rng):
    zone_fracs = {'Center': 0.17, 'Transition': 0.41, 'Boundary': 0.42}
    zone_counts = {}
    remaining = length
    for z in ['Center', 'Transition']:
        zone_counts[z] = max(2, round(length * zone_fracs[z]))
        remaining -= zone_counts[z]
    zone_counts['Boundary'] = max(2, remaining)

    composition = {}
    for z in ZONES:
        z_total = zone_counts[z]
        probs = [signature[(z, t)] for t in TYPES]
        s = sum(probs) or 1.0
        probs = [p / s for p in probs]
        counts_arr = rng.multinomial(z_total, probs)
        counts = {t: int(c) for t, c in zip(TYPES, counts_arr)}
        composition[z] = counts
    return composition, zone_counts


def generate_sequence(length, signature, rng):
    composition, zone_counts = generate_composition(length, signature, rng)
    aa_pools = {'Gon': GON_AAS, 'Gam': GAM_AAS, 'Ri': RI_AAS, 'Geon': GEON_AAS}

    zone_sequences = {}
    zone_residues = {}

    for z in ZONES:
        seq = []
        residues = []
        for t in TYPES:
            n = composition[z][t]
            pool = aa_pools[t]
            aas = [pool[rng.integers(0, len(pool))] for _ in range(n)]
            for aa in aas:
                seq.append(aa)
                residues.append({'aa': aa, 'type': t, 'zone': z})
        rng.shuffle(seq)
        indices = list(range(len(residues)))
        rng.shuffle(indices)
        residues = [residues[i] for i in indices]
        zone_sequences[z] = seq
        zone_residues[z] = residues

    full_sequence = zone_sequences['Center'] + zone_sequences['Transition'] + zone_sequences['Boundary']
    full_residues = zone_residues['Center'] + zone_residues['Transition'] + zone_residues['Boundary']
    return ''.join(full_sequence), full_residues, composition, zone_counts


# ============================================================
# STAGE 3: 소거 필터
# ============================================================
def elimination_filter(sequence, residues, composition, zone_counts, target_signature):
    violations = []
    total = len(sequence)
    type_counts = defaultdict(int)
    for aa in sequence:
        type_counts[TYPE_MAP[aa]] += 1
    type_ratios = {t: type_counts[t] / total for t in TYPES}

    targets = {'Gon': 0.35, 'Gam': 0.15, 'Ri': 0.25, 'Geon': 0.20}
    tolerance = 0.07
    for t in TYPES:
        if abs(type_ratios[t] - targets[t]) > tolerance:
            violations.append(f"조성소거: {t} = {type_ratios[t]:.1%}")

    center_residues = [r for r in residues if r['zone'] == 'Center']
    if center_residues:
        geon_in_center = sum(1 for r in center_residues if r['type'] == 'Geon') / len(center_residues)
        if geon_in_center > 0.30:
            violations.append(f"Zone소거: Center 건 = {geon_in_center:.1%}")

    boundary_residues = [r for r in residues if r['zone'] == 'Boundary']
    if boundary_residues:
        gon_in_boundary = sum(1 for r in boundary_residues if r['type'] == 'Gon') / len(boundary_residues)
        if gon_in_boundary > 0.45:
            violations.append(f"Zone소거: Boundary 곤 = {gon_in_boundary:.1%}")

    adjacent_gon_geon = 0
    total_pairs = len(sequence) - 1
    for i in range(total_pairs):
        t1 = TYPE_MAP[sequence[i]]
        t2 = TYPE_MAP[sequence[i + 1]]
        if (t1 == 'Gon' and t2 == 'Geon') or (t1 == 'Geon' and t2 == 'Gon'):
            adjacent_gon_geon += 1
    if total_pairs > 0:
        adj_ratio = adjacent_gon_geon / total_pairs
        expected = 2 * type_ratios.get('Gon', 0.35) * type_ratios.get('Geon', 0.20)
        oe_ratio = adj_ratio / expected if expected > 0 else 0
        if oe_ratio > 1.5:
            violations.append(f"패턴소거: 곤-건 O/E = {oe_ratio:.2f}")

    actual_sig = {}
    for z in ZONES:
        z_res = [r for r in residues if r['zone'] == z]
        z_total = len(z_res) if z_res else 1
        for t in TYPES:
            actual_sig[(z, t)] = sum(1 for r in z_res if r['type'] == t) / z_total

    actual_antag = (
        (actual_sig[('Center','Gon')] - actual_sig[('Boundary','Gon')]) -
        (actual_sig[('Center','Geon')] - actual_sig[('Boundary','Geon')])
    )
    target_antag = (
        (target_signature[('Center','Gon')] - target_signature[('Boundary','Gon')]) -
        (target_signature[('Center','Geon')] - target_signature[('Boundary','Geon')])
    )
    if abs(actual_antag - target_antag) > 0.15:
        violations.append(f"상극축소거: Δ={abs(actual_antag-target_antag):.3f}")

    passed = len(violations) == 0
    return passed, violations, type_ratios, actual_sig, actual_antag


# ============================================================
# STAGE 4: 성장 3D 시뮬레이션
# ============================================================
def growth_simulation(residues, zone_counts, n_cycles=5, gravity_scale=1.0):
    rng = np.random.default_rng(42)
    n = len(residues)
    coords = np.zeros((n, 3))
    max_radius = (n / 4) ** (1/3) * 3.8

    for z_idx, zone_name in enumerate(ZONES):
        zone_res = [i for i, r in enumerate(residues) if r['zone'] == zone_name]
        n_zone = len(zone_res)
        if zone_name == 'Center':
            r_min, r_max = 0, max_radius * CENTER_THRESH
        elif zone_name == 'Transition':
            r_min, r_max = max_radius * CENTER_THRESH, max_radius * TRANS_THRESH
        else:
            r_min, r_max = max_radius * TRANS_THRESH, max_radius

        for j, res_idx in enumerate(zone_res):
            golden_angle = math.pi * (3 - math.sqrt(5))
            theta = golden_angle * j
            phi_angle = math.acos(1 - 2 * (j + 0.5) / max(n_zone, 1))
            r = rng.uniform(r_min, r_max)
            coords[res_idx] = [
                r * math.sin(phi_angle) * math.cos(theta),
                r * math.sin(phi_angle) * math.sin(theta),
                r * math.cos(phi_angle)
            ]

    gravity = {'Gon': 0.8*gravity_scale, 'Gam': 0.3*gravity_scale,
               'Ri': 0.2*gravity_scale, 'Geon': -0.4*gravity_scale}
    active = np.ones(n, dtype=bool)

    for cycle in range(n_cycles):
        center = coords.mean(axis=0)
        new_coords = coords.copy()
        for i in range(n):
            ti = residues[i]['type']
            dists = np.linalg.norm(coords - coords[i], axis=1)
            neighbors = np.where((dists > 0.1) & (dists < 12.0))[0]
            if len(neighbors) == 0:
                continue
            force = np.zeros(3)
            for j in neighbors:
                tj = residues[j]['type']
                aff = AFFINITY[(ti, tj)]
                direction = coords[j] - coords[i]
                dist = np.linalg.norm(direction)
                if dist > 0.1:
                    force += aff * direction / (dist ** 2)
                    if (ti == 'Gon' and tj == 'Geon') or (ti == 'Geon' and tj == 'Gon'):
                        force -= 2.0 * direction / (dist ** 2)
            to_center = center - coords[i]
            dist_to_center = np.linalg.norm(to_center)
            if dist_to_center > 0.1:
                force += gravity[ti] * to_center / dist_to_center
            dist_from_center = np.linalg.norm(coords[i] - center)
            if dist_from_center > max_radius * 1.2:
                force += 0.5 * (center - coords[i]) / dist_from_center
            step = 0.2
            if np.linalg.norm(force) > 0:
                new_coords[i] = coords[i] + step * force / np.linalg.norm(force)
        coords = new_coords
        for _ in range(2):
            for i in range(n):
                dists = np.linalg.norm(coords - coords[i], axis=1)
                too_close = np.where((dists > 0.01) & (dists < 2.0))[0]
                for j in too_close:
                    diff = coords[i] - coords[j]
                    dist = np.linalg.norm(diff)
                    if dist > 0.01:
                        push = (2.0 - dist) / 2 * diff / dist
                        coords[i] += push * 0.3
                        coords[j] -= push * 0.3
    return coords, active


# ============================================================
# STAGE 5: 3D → 12칸 재분석
# ============================================================
def analyze_3d_structure(coords, residues, active):
    active_idx = np.where(active)[0]
    active_coords = coords[active_idx]
    active_residues = [residues[i] for i in active_idx]
    if len(active_coords) < 5:
        return None
    center = active_coords.mean(axis=0)
    distances = np.linalg.norm(active_coords - center, axis=1)
    max_dist = distances.max()
    if max_dist < 0.01:
        return None
    normalized = distances / max_dist

    zones_3d = []
    for nd in normalized:
        if nd < CENTER_THRESH:
            zones_3d.append('Center')
        elif nd < TRANS_THRESH:
            zones_3d.append('Transition')
        else:
            zones_3d.append('Boundary')

    zone_index_map = {z: [i for i, zn in enumerate(zones_3d) if zn == z] for z in ZONES}
    if len(zone_index_map['Center']) == 0 or len(zone_index_map['Boundary']) == 0:
        return None

    matrix_3d = {}
    for z in ZONES:
        z_indices = zone_index_map[z]
        z_total = len(z_indices) if z_indices else 1
        for t in TYPES:
            count = sum(1 for i in z_indices if active_residues[i]['type'] == t)
            matrix_3d[(z, t)] = count / z_total

    antagonism_3d = (
        (matrix_3d[('Center','Gon')] - matrix_3d[('Boundary','Gon')]) -
        (matrix_3d[('Center','Geon')] - matrix_3d[('Boundary','Geon')])
    )
    center_types = {t: matrix_3d[('Center', t)] for t in TYPES}
    gon_is_center_max = max(center_types, key=center_types.get) == 'Gon'
    zone_counts_3d = {z: sum(1 for zn in zones_3d if zn == z) for z in ZONES}

    return {
        'matrix': matrix_3d, 'antagonism': antagonism_3d,
        'gon_center_max': gon_is_center_max,
        'n_active': len(active_idx), 'n_original': len(residues),
        'zone_counts': zone_counts_3d, 'zones': zones_3d,
    }


# ============================================================
# STAGE 6: 자기검증
# ============================================================
def self_verification(target_antagonism, input_signature, result_3d, length=None):
    if result_3d is None:
        return {'closed': False, 'reason': '3D 분석 실패'}

    delta_max = 0.20
    rmse_max = 0.15
    if length is not None and length < 120:
        delta_max = delta_max + (120 - length) * 0.002
        delta_max = min(delta_max, 0.28)

    output_antagonism = result_3d['antagonism']
    delta = abs(target_antagonism - output_antagonism)

    rmse = 0
    n_cells = 0
    for z in ZONES:
        for t in TYPES:
            diff = input_signature[(z, t)] - result_3d['matrix'].get((z, t), 0.0)
            rmse += diff ** 2
            n_cells += 1
    rmse = math.sqrt(rmse / n_cells)

    gon_grad_in = input_signature[('Center','Gon')] - input_signature[('Boundary','Gon')]
    gon_grad_out = result_3d['matrix'][('Center','Gon')] - result_3d['matrix'][('Boundary','Gon')]
    gon_match = (gon_grad_in > 0) == (gon_grad_out > 0)

    geon_grad_in = input_signature[('Center','Geon')] - input_signature[('Boundary','Geon')]
    geon_grad_out = result_3d['matrix'][('Center','Geon')] - result_3d['matrix'][('Boundary','Geon')]
    geon_match = (geon_grad_in < 0) == (geon_grad_out < 0)

    closed = (delta < delta_max and rmse < rmse_max and gon_match and geon_match and result_3d['gon_center_max'])

    return {
        'closed': closed,
        'target_antagonism': target_antagonism,
        'output_antagonism': output_antagonism,
        'delta_antagonism': delta,
        'rmse_12cell': rmse,
        'gon_gradient_match': gon_match,
        'geon_gradient_match': geon_match,
        'gon_center_max': result_3d['gon_center_max'],
        'survival_rate': result_3d['n_active'] / result_3d['n_original'],
    }

print("✅ Creator Engine Stage 1-6 정의 완료")


## 3. Creator Engine 실행

In [ ]:
def run_creator_engine(length, target_antagonism, n_candidates=10, verbose=True):
    """UFoE Creator Engine 전체 파이프라인. FASTA 출력 포함."""
    signature = generate_signature(target_antagonism)
    input_antag = (
        (signature[('Center','Gon')] - signature[('Boundary','Gon')]) -
        (signature[('Center','Geon')] - signature[('Boundary','Geon')])
    )

    if verbose:
        print(f"🧬 UFoE Creator Engine: L={length}, A={target_antagonism:.3f}")
        print(f"  시그니처 상극축: {input_antag:.4f}")

    results = []
    for cid in range(n_candidates):
        rng = np.random.default_rng(42 + cid * 7)
        sequence, residues, composition, zone_counts = generate_sequence(length, signature, rng)
        passed, violations, type_ratios, actual_sig, actual_antag = (
            elimination_filter(sequence, residues, composition, zone_counts, signature)
        )
        if not passed:
            results.append({'id': cid, 'passed_filter': False, 'violations': violations, 'sequence': sequence})
            continue

        n_cycles = 60 if length < 120 else 40
        gravity_scale = 1.15 if length < 120 else 1.0
        coords, active = growth_simulation(residues, zone_counts, n_cycles=n_cycles, gravity_scale=gravity_scale)
        result_3d = analyze_3d_structure(coords, residues, active)
        verification = self_verification(target_antagonism, signature, result_3d, length=length)

        results.append({
            'id': cid, 'passed_filter': True, 'sequence': sequence,
            'type_ratios': type_ratios, 'verification': verification, 'result_3d': result_3d,
        })

    # 최선 후보 선택
    closed = [r for r in results if r['passed_filter'] and r.get('verification', {}).get('closed', False)]
    if closed:
        best = min(closed, key=lambda r: r['verification']['delta_antagonism'])
    else:
        passed_r = [r for r in results if r['passed_filter'] and r.get('verification')]
        best = min(passed_r, key=lambda r: r['verification']['delta_antagonism']) if passed_r else None

    if verbose and best:
        v = best['verification']
        print(f"  ✅ 최선 후보 #{best['id']}: A={v['output_antagonism']:.4f} (Δ={v['delta_antagonism']:.4f})")
        print(f"     곤↓={'✓' if v['gon_gradient_match'] else '✗'}  건↑={'✓' if v['geon_gradient_match'] else '✗'}  곤C={'✓' if v['gon_center_max'] else '✗'}")
        print(f"     순환: {'✅ 닫힘' if v['closed'] else '⚠ 미닫힘'}")

    return results, signature, best


def save_fasta(sequences_dict, filepath):
    """서열 딕셔너리를 FASTA 파일로 저장."""
    with open(filepath, 'w') as f:
        for name, seq in sequences_dict.items():
            f.write(f">{name}\n")
            for i in range(0, len(seq), 60):
                f.write(seq[i:i+60] + "\n")
    print(f"  💾 FASTA 저장: {filepath} ({len(sequences_dict)}개 서열)")

print("✅ 파이프라인 함수 정의 완료")


## 4. 서열 생성 실행

3가지 단백질 유형을 생성합니다:
- **효소형** (A=0.30, L=150): 가장 높은 자기검증 통과율
- **구형** (A=0.50, L=200): 강한 곤-건 분리
- **고안정 구형** (A=0.55, L=150): 극단적 상극축


In [ ]:
# ═══════════════════════════════════════
# 서열 생성 + FASTA 저장
# ═══════════════════════════════════════

test_cases = [
    (150, 0.30, "enzyme"),
    (200, 0.50, "globular"),
    (150, 0.55, "hyperstable"),
    (250, 0.20, "multidomain"),
]

all_results = {}
all_signatures = {}
all_best = {}
fasta_sequences = {}

for length, antag, label in test_cases:
    print(f"\n{'━' * 60}")
    results, signature, best = run_creator_engine(length, antag, n_candidates=10)
    all_results[label] = results
    all_signatures[label] = signature
    all_best[label] = best

    if best:
        seq_name = f"UFoE_{label}_L{length}_A{antag:.2f}"
        fasta_sequences[seq_name] = best['sequence']

        # 12칸 매트릭스 출력
        r3d = best['result_3d']
        print(f"  3D 매트릭스:")
        print(f"    {'Zone':>12}  {'Gon':>7}  {'Gam':>7}  {'Ri':>7}  {'Geon':>7}")
        for z in ZONES:
            row = [r3d['matrix'][(z, t)] for t in TYPES]
            print(f"    {z:>12}  {row[0]:>6.1%}  {row[1]:>6.1%}  {row[2]:>6.1%}  {row[3]:>6.1%}")

# FASTA 저장
os.makedirs("output", exist_ok=True)
save_fasta(fasta_sequences, "output/ufoe_sequences.fasta")

print(f"\n{'═' * 60}")
print(f"  총 {len(fasta_sequences)}개 서열 생성 완료")
print(f"{'═' * 60}")


## 5. ESMFold 구조 예측

> ⚠ **GPU 필요**: Runtime > Change runtime type > T4 GPU

ESMFold는 단일 서열만으로 3D 구조를 예측합니다. MSA 불필요.
UFoE의 곤감리건 분류를 전혀 모르는 독립적 AI 모델입니다.


In [ ]:
# ESMFold 설치 및 로드
!pip install -q fair-esm

import torch
import esm

print("🔄 ESMFold 모델 로딩 중... (첫 실행 시 ~3분)")
model = esm.pretrained.esmfold_v1()
model = model.eval()

# GPU로 이동
if torch.cuda.is_available():
    model = model.cuda()
    print(f"✅ ESMFold 로드 완료 (GPU: {torch.cuda.get_device_name(0)})")
else:
    print("⚠ CPU 모드 — 매우 느릴 수 있음")


In [ ]:
# ═══════════════════════════════════════
# ESMFold 구조 예측 실행
# ═══════════════════════════════════════

def predict_structure(model, sequence, name):
    """ESMFold로 구조 예측. PDB 문자열 + pLDDT 반환."""
    print(f"  🔬 {name} (L={len(sequence)})...", end=" ")
    start = time.time()

    with torch.no_grad():
        output = model.infer_pdb(sequence)

    elapsed = time.time() - start

    # pLDDT 추출 (B-factor 칼럼)
    plddt_values = []
    for line in output.split("\n"):
        if line.startswith("ATOM"):
            try:
                plddt_values.append(float(line[60:66].strip()))
            except:
                pass

    mean_plddt = np.mean(plddt_values) if plddt_values else 0
    print(f"완료 ({elapsed:.1f}s, pLDDT={mean_plddt:.1f})")

    return output, mean_plddt, plddt_values


# 예측 실행
esm_results = {}

for name, seq in fasta_sequences.items():
    pdb_str, mean_plddt, plddt_vals = predict_structure(model, seq, name)

    # PDB 저장
    pdb_path = f"output/{name}.pdb"
    with open(pdb_path, 'w') as f:
        f.write(pdb_str)

    esm_results[name] = {
        'pdb_str': pdb_str,
        'pdb_path': pdb_path,
        'mean_plddt': mean_plddt,
        'plddt_values': plddt_vals,
        'sequence': seq,
    }

print(f"\n✅ {len(esm_results)}개 구조 예측 완료")
for name, r in esm_results.items():
    print(f"  {name}: pLDDT = {r['mean_plddt']:.1f}")


## 6. ESMFold 출력 → 12칸 시그니처 측정

ESMFold가 예측한 3D 구조에서 각 잔기의 Cα 좌표를 추출하고,
중심으로부터의 거리를 기반으로 Zone을 할당한 뒤, 12칸 시그니처를 계산합니다.

이것이 **외부 검증의 핵심 단계**입니다.


In [ ]:
def extract_ca_from_pdb(pdb_str):
    """PDB 문자열에서 Cα 좌표와 잔기 정보 추출."""
    cas = []
    for line in pdb_str.split("\n"):
        if line.startswith("ATOM") and line[12:16].strip() == "CA":
            x = float(line[30:38])
            y = float(line[38:46])
            z = float(line[46:54])
            resname = line[17:20].strip()
            resnum = int(line[22:26])
            bfactor = float(line[60:66].strip())  # pLDDT
            cas.append({
                'coord': np.array([x, y, z]),
                'resname': resname,
                'resnum': resnum,
                'plddt': bfactor,
            })
    return cas


# 3글자 → 1글자 변환
AA3_TO_1 = {
    'ALA':'A','VAL':'V','ILE':'I','LEU':'L','PHE':'F','TRP':'W','MET':'M',
    'CYS':'C','GLY':'G','PRO':'P',
    'SER':'S','THR':'T','ASN':'N','GLN':'Q','HIS':'H','TYR':'Y',
    'LYS':'K','ARG':'R','ASP':'D','GLU':'E',
}


def compute_signature_from_pdb(pdb_str):
    """ESMFold PDB에서 12칸 시그니처 계산."""
    cas = extract_ca_from_pdb(pdb_str)
    if len(cas) < 10:
        return None, None

    coords = np.array([ca['coord'] for ca in cas])
    center = coords.mean(axis=0)
    distances = np.linalg.norm(coords - center, axis=1)
    max_dist = distances.max()

    if max_dist < 0.01:
        return None, None

    normalized = distances / max_dist

    # Zone 할당
    for i, ca in enumerate(cas):
        nd = normalized[i]
        if nd < CENTER_THRESH:
            ca['zone'] = 'Center'
        elif nd < TRANS_THRESH:
            ca['zone'] = 'Transition'
        else:
            ca['zone'] = 'Boundary'

        # 타입 할당
        aa1 = AA3_TO_1.get(ca['resname'], '?')
        ca['aa1'] = aa1
        ca['type'] = TYPE_MAP.get(aa1, 'Unknown')

    # 12칸 매트릭스
    matrix = {}
    zone_counts = {}
    for z in ZONES:
        z_cas = [ca for ca in cas if ca['zone'] == z and ca['type'] != 'Unknown']
        zone_counts[z] = len(z_cas)
        z_total = len(z_cas) if z_cas else 1
        for t in TYPES:
            count = sum(1 for ca in z_cas if ca['type'] == t)
            matrix[(z, t)] = count / z_total

    # 상극축
    if zone_counts.get('Center', 0) == 0 or zone_counts.get('Boundary', 0) == 0:
        return None, None

    antagonism = (
        (matrix[('Center','Gon')] - matrix[('Boundary','Gon')]) -
        (matrix[('Center','Geon')] - matrix[('Boundary','Geon')])
    )

    gon_center_max = max((t for t in TYPES), key=lambda t: matrix[('Center', t)]) == 'Gon'
    gon_grad = matrix[('Center','Gon')] > matrix[('Boundary','Gon')]
    geon_grad = matrix[('Center','Geon')] < matrix[('Boundary','Geon')]

    result = {
        'matrix': matrix,
        'antagonism': antagonism,
        'zone_counts': zone_counts,
        'gon_center_max': gon_center_max,
        'gon_gradient': gon_grad,
        'geon_gradient': geon_grad,
        'n_residues': len(cas),
        'mean_plddt': np.mean([ca['plddt'] for ca in cas]),
    }

    return result, cas

print("✅ PDB 시그니처 분석 함수 정의 완료")


## 7. 핵심 검증: 입력 시그니처 vs ESMFold 출력 비교

> **이것이 UFoE 외부 검증의 핵심입니다.**
>
> Creator Engine이 의도한 12칸 기울기가
> ESMFold라는 독립적 AI 모델의 3D 예측에서도 보존되는가?


In [ ]:
# ═══════════════════════════════════════
# 입력 vs ESMFold 출력 비교
# ═══════════════════════════════════════

comparison_results = []

print(f"{'═' * 80}")
print(f"  UFoE vs ESMFold 시그니처 비교")
print(f"{'═' * 80}")

for name, esm_r in esm_results.items():
    # 어떤 테스트 케이스인지 매칭
    label = name.split('_')[1]  # enzyme, globular, etc.

    input_sig = all_signatures.get(label)
    if input_sig is None:
        continue

    # ESMFold PDB에서 시그니처 추출
    esm_sig, cas = compute_signature_from_pdb(esm_r['pdb_str'])
    if esm_sig is None:
        print(f"  ⚠ {name}: PDB 분석 실패")
        continue

    # 입력 상극축
    input_antag = (
        (input_sig[('Center','Gon')] - input_sig[('Boundary','Gon')]) -
        (input_sig[('Center','Geon')] - input_sig[('Boundary','Geon')])
    )

    # 비교
    delta_antag = abs(input_antag - esm_sig['antagonism'])

    # RMSE
    rmse = 0
    for z in ZONES:
        for t in TYPES:
            diff = input_sig[(z, t)] - esm_sig['matrix'][(z, t)]
            rmse += diff ** 2
    rmse = math.sqrt(rmse / 12)

    result = {
        'name': name,
        'label': label,
        'input_antagonism': input_antag,
        'esm_antagonism': esm_sig['antagonism'],
        'delta_antagonism': delta_antag,
        'rmse_12cell': rmse,
        'gon_gradient': esm_sig['gon_gradient'],
        'geon_gradient': esm_sig['geon_gradient'],
        'gon_center_max': esm_sig['gon_center_max'],
        'mean_plddt': esm_r['mean_plddt'],
        'input_sig': input_sig,
        'esm_sig': esm_sig,
    }
    comparison_results.append(result)

    # 출력
    print(f"\n  📊 {name}")
    print(f"  pLDDT: {esm_r['mean_plddt']:.1f} {'(신뢰)' if esm_r['mean_plddt'] > 70 else '(낮음)'}")
    print(f"  상극축: 입력 {input_antag:.4f} → ESMFold {esm_sig['antagonism']:.4f} (Δ={delta_antag:.4f})")
    print(f"  12칸 RMSE: {rmse:.4f}")
    print(f"  곤↓: {'✓' if esm_sig['gon_gradient'] else '✗'}  건↑: {'✓' if esm_sig['geon_gradient'] else '✗'}  곤Center: {'✓' if esm_sig['gon_center_max'] else '✗'}")

    # 12칸 비교 테이블
    print(f"\n  {'':>12}  {'─ 입력(UFoE) ─':>28}  {'─ ESMFold 출력 ─':>28}")
    print(f"  {'Zone':>12}  {'Gon':>6} {'Gam':>6} {'Ri':>6} {'Geon':>6}  {'Gon':>6} {'Gam':>6} {'Ri':>6} {'Geon':>6}")
    for z in ZONES:
        i_row = [input_sig[(z, t)] for t in TYPES]
        e_row = [esm_sig['matrix'][(z, t)] for t in TYPES]
        print(f"  {z:>12}  {i_row[0]:>5.1%} {i_row[1]:>5.1%} {i_row[2]:>5.1%} {i_row[3]:>5.1%}  "
              f"{e_row[0]:>5.1%} {e_row[1]:>5.1%} {e_row[2]:>5.1%} {e_row[3]:>5.1%}")

# 종합 요약
print(f"\n\n{'═' * 80}")
print(f"  종합 결과")
print(f"{'═' * 80}")
n_gon_grad = sum(1 for r in comparison_results if r['gon_gradient'])
n_geon_grad = sum(1 for r in comparison_results if r['geon_gradient'])
n_gon_center = sum(1 for r in comparison_results if r['gon_center_max'])
n_total = len(comparison_results)

print(f"  곤 기울기 보존 (Center > Boundary): {n_gon_grad}/{n_total}")
print(f"  건 기울기 보존 (Center < Boundary): {n_geon_grad}/{n_total}")
print(f"  곤 Center 최다 보존:               {n_gon_center}/{n_total}")
print(f"  평균 상극축 Δ: {np.mean([r['delta_antagonism'] for r in comparison_results]):.4f}")
print(f"  평균 12칸 RMSE: {np.mean([r['rmse_12cell'] for r in comparison_results]):.4f}")
print(f"  평균 pLDDT: {np.mean([r['mean_plddt'] for r in comparison_results]):.1f}")


## 8. 시각화

In [ ]:
# ═══════════════════════════════════════
# 12칸 시그니처 비교 히트맵
# ═══════════════════════════════════════

n_cases = len(comparison_results)
if n_cases == 0:
    print("⚠ 비교 결과 없음")
else:
    fig, axes = plt.subplots(n_cases, 3, figsize=(16, 4 * n_cases))
    if n_cases == 1:
        axes = [axes]

    for idx, r in enumerate(comparison_results):
        # 입력 매트릭스
        input_data = np.array([[r['input_sig'][(z, t)] for t in TYPES] for z in ZONES])
        # ESMFold 매트릭스
        esm_data = np.array([[r['esm_sig']['matrix'][(z, t)] for t in TYPES] for z in ZONES])
        # 차이
        diff_data = esm_data - input_data

        for col, (data, title, cmap, vmin, vmax) in enumerate([
            (input_data, f"입력 (UFoE)", 'Blues', 0, 0.5),
            (esm_data, f"ESMFold 출력", 'Blues', 0, 0.5),
            (diff_data, f"차이 (ESM - UFoE)", 'RdBu_r', -0.15, 0.15),
        ]):
            ax = axes[idx][col]
            sns.heatmap(data, ax=ax, annot=True, fmt='.1%', cmap=cmap,
                       vmin=vmin, vmax=vmax,
                       xticklabels=TYPES, yticklabels=ZONES if col == 0 else [])
            ax.set_title(f"{r['label']}\n{title}", fontsize=10)

    plt.suptitle("UFoE 입력 vs ESMFold 출력 — 12칸 시그니처 비교", fontsize=14, y=1.02)
    plt.tight_layout()
    plt.savefig("output/signature_comparison.png", dpi=150, bbox_inches='tight')
    plt.show()
    print("💾 output/signature_comparison.png 저장")


In [ ]:
# ═══════════════════════════════════════
# 상극축 비교 차트
# ═══════════════════════════════════════

if comparison_results:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

    # 상극축 입력 vs 출력
    labels = [r['label'] for r in comparison_results]
    input_a = [r['input_antagonism'] for r in comparison_results]
    esm_a = [r['esm_antagonism'] for r in comparison_results]
    plddts = [r['mean_plddt'] for r in comparison_results]

    x = np.arange(len(labels))
    w = 0.35
    bars1 = ax1.bar(x - w/2, input_a, w, label='UFoE 입력', color='#2196F3', alpha=0.8)
    bars2 = ax1.bar(x + w/2, esm_a, w, label='ESMFold 출력', color='#F44336', alpha=0.8)
    ax1.set_xlabel('단백질 유형')
    ax1.set_ylabel('상극축')
    ax1.set_title('상극축: UFoE 입력 vs ESMFold 출력')
    ax1.set_xticks(x)
    ax1.set_xticklabels(labels, rotation=30)
    ax1.legend()
    ax1.axhline(y=BASELINE_ANTAGONISM, color='gray', linestyle='--', alpha=0.5, label='PDB 평균')

    # 곤/건 기울기 보존 + pLDDT
    for i, r in enumerate(comparison_results):
        markers = ""
        if r['gon_gradient']: markers += "곤↓✓ "
        else: markers += "곤↓✗ "
        if r['geon_gradient']: markers += "건↑✓"
        else: markers += "건↑✗"
        ax1.text(i, max(input_a[i], esm_a[i]) + 0.02, markers, ha='center', fontsize=8)

    # pLDDT 바
    colors = ['#4CAF50' if p > 70 else '#FF9800' if p > 50 else '#F44336' for p in plddts]
    ax2.bar(labels, plddts, color=colors, alpha=0.8)
    ax2.axhline(y=70, color='green', linestyle='--', alpha=0.5, label='pLDDT=70 (신뢰)')
    ax2.set_ylabel('mean pLDDT')
    ax2.set_title('ESMFold 접힘 신뢰도')
    ax2.set_xticklabels(labels, rotation=30)
    ax2.legend()
    ax2.set_ylim(0, 100)

    plt.tight_layout()
    plt.savefig("output/antagonism_comparison.png", dpi=150, bbox_inches='tight')
    plt.show()
    print("💾 output/antagonism_comparison.png 저장")


## 9. 결과 저장 및 요약

모든 결과를 JSON과 CSV로 저장합니다.


In [ ]:
# 결과 저장
summary_data = []
for r in comparison_results:
    summary_data.append({
        'name': r['name'],
        'label': r['label'],
        'input_antagonism': round(r['input_antagonism'], 4),
        'esm_antagonism': round(r['esm_antagonism'], 4),
        'delta_antagonism': round(r['delta_antagonism'], 4),
        'rmse_12cell': round(r['rmse_12cell'], 4),
        'gon_gradient_preserved': r['gon_gradient'],
        'geon_gradient_preserved': r['geon_gradient'],
        'gon_center_max_preserved': r['gon_center_max'],
        'mean_plddt': round(r['mean_plddt'], 1),
    })

# JSON
with open("output/ufoe_esmfold_validation.json", 'w') as f:
    json.dump(summary_data, f, indent=2, ensure_ascii=False)

# CSV
df = pd.DataFrame(summary_data)
df.to_csv("output/ufoe_esmfold_validation.csv", index=False)

print("💾 결과 저장 완료:")
print("  output/ufoe_esmfold_validation.json")
print("  output/ufoe_esmfold_validation.csv")
print("  output/ufoe_sequences.fasta")
print("  output/*.pdb (ESMFold 구조)")
print("  output/*.png (시각화)")

print(f"\n{'═' * 80}")
print(f"  🧬 UFoE × ESMFold 외부 검증 완료")
print(f"{'═' * 80}")
print(f"\n  핵심 질문: 'UFoE가 설계한 12칸 기울기가 ESMFold에서 보존되는가?'")
print(f"  → 위 결과 확인")
print(f"\n  pLDDT > 70이면 ESMFold가 '이 서열은 접힌다'고 판단한 것입니다.")
print(f"  기울기가 보존되면 UFoE의 물리적 원리가 독립 검증된 것입니다.")
